In [ ]:
# MNIST - Neural Network
# layer 별로 weight가 있는데 이걸 각각 gradient를 구해야하는지 gradient 계산이 애매
# 각 layer의 weight들을 하나로 묶어서 계산은 안되나도 고민
import numpy as np
import tensorflow as tf

(images_train, labels_train), (images_test, labels_test) = tf.keras.datasets.mnist.load_data()

class NeuralNetwork:
    def __init__(self, features_train, labels_train):
        self.features, self.labels = self.preprocess(features_train, labels_train)
        self.weights = np.random.rand(len(features_train[0]))
    def preprocess(self, features_before, labels_before):
        features = np.reshape(features_before, (len(features_before), 784))
        features = self.normalize(features)
        labels = self.one_hot_encoding(labels_before)
        return np.array(features), np.array(labels)
    def normalize(self, data):
        return (data - np.min(data)) / (np.max(data) - np.min(data))
    def one_hot_encoding(self, data):
        encoded = [([0] * (np.max(data) - np.min(data) + 1)) for i in range(len(data))]
        for idx in range(len(data)):
            encoded[idx][data[idx]] = 1
        return encoded
    def hypothesis(self, features):
        
    def sigmoid(self, z):
        return 1 / (1 + np.exp(-1 * z))
    def cost(self, hypothesis, labels):
        return np.multiply(labels, np.log(hypothesis)) + np.multiply(1 - labels, np.log(1 - hypothesis))
    def grad(self, features, hypothesis, labels):

    def gradient_descent(self, learning_rate, EPOCHS, batch_size):
        for step in range(EPOCHS + 1):
            for batch_idx in range(len(self.features) / batch_size):
                features = self.features[batch_idx * batch_size : (batch_idx + 1) * batch_size]
                hypothesis = self.hypothesis(features)
                grad = self.grad(features, hypothesis, self.labels)
                self.weights = self.weights # -? +?
                cost = self.cost(hypothesis, self.labels)
            if step % (EPOCHS / 10) == 0:
                print("{:6} : cost {:10.4f}".format(step, cost))
model = NeuralNetwork(images_train, labels_train)
print(model.labels)

[[0 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 1 0]]


In [8]:
# 2 Layer NN(ReLU, Softmax) with no batch
import sys
import numpy as np
import tensorflow as tf

(images_train, labels_train), (images_test, labels_test) = tf.keras.datasets.mnist.load_data()
images_train = np.reshape(images_train / 255.0, (len(images_train), len(images_train[0]) * len(images_train[0][0]))).T
images_test = np.reshape(images_test / 255.0, (len(images_test), len(images_test[0]) * len(images_test[0][0]))).T

def one_hot_encoding(data):
    encoded = np.zeros((len(data), max(data) - min(data) + 1))
    for idx in range(len(data)):
        encoded[idx][data[idx]] = 1
    return encoded.T

labels_train, labels_test = one_hot_encoding(labels_train), one_hot_encoding(labels_test)

class NeuralNetwork_2():
    def __init__(self, n0, n1, n2):
        self.weight1 = np.random.rand(n1, n0)
        self.weight2 = np.random.rand(n2, n1)
        self.bias1 = np.random.rand(n1)
        self.bias2 = np.random.rand(n2)
    def linear_hypo(self, w, x, b):
        return (np.matmul(w, x).T + b).T
    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))
    def relu(self, z):
        return np.maximum(z, 0)
    def softmax(self, z):
        return np.exp(z) / np.exp(z).sum(axis=0)
    def hypothesis(self, func, lh):
        if func == 'sigmoid':
            return self.sigmoid(lh)
        elif func == 'relu':
            return self.relu(lh)
        elif func == 'softmax':
            return self.softmax(lh)
        else:
            sys.exit('Error in hypothesis: There is no {} function'.format(func))
    def cost(self, y, hypo):
        # cross entropy
        sum = np.multiply(y, np.log(hypo)).sum(axis=0)
        return -np.average(sum)
    def train(self, features, labels, learning_rate, EPOCHS):
        for iter in range(EPOCHS + 1):
            # layer1: relu, layer2: softmax, cost: cross-entropy
            z1 = self.linear_hypo(self.weight1, features, self.bias1)
            layer1 = self.hypothesis('relu', z1)
            z2 = self.linear_hypo(self.weight2, layer1, self.bias2)
            layer2 = self.hypothesis('softmax', z2)
            # gradient
            dz2 = layer2 - labels
            dw2 = np.matmul(dz2, layer1.T)
            db2 = dz2.sum(axis=1)

            dz1 = np.multiply(np.matmul(self.weight2.T, dz2), np.where(z1 > 0, 1, 0)) # relu gradient: if x > 0: 1 else: 0 => np.where(z1 > 0, 1, 0)
            dw1 = np.matmul(dz1, features.T)
            db1 = dz1.sum(axis=1)

            self.weight2 = self.weight2 - dw2 * learning_rate
            self.weight1 = self.weight1 - dw1 * learning_rate
            self.bias2 = self.bias2 - db2 * learning_rate
            self.bias1 = self.bias1 - db1 * learning_rate

            error = self.cost(labels, layer2)
            if iter % (EPOCHS / 10) == 0:
                print("iter: {:4} error: {:10.4f}".format(iter, error))
model = NeuralNetwork_2(len(images_train), 5, 10)
model.train(images_train, labels_train, 0.00001, 10)


iter:    0 error:    29.8609


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: RuntimeWarning: overflow encountered in exp
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: RuntimeWarning: invalid value encountered in multiply


iter:    1 error:        nan
iter:    2 error:        nan
iter:    3 error:        nan
iter:    4 error:        nan
iter:    5 error:        nan
iter:    6 error:        nan
iter:    7 error:        nan
iter:    8 error:        nan
iter:    9 error:        nan
iter:   10 error:        nan


In [22]:
# 2 Layer NN(ReLU, Softmax) with batch
import sys
import numpy as np
import tensorflow as tf

(images_train, labels_train), (images_test, labels_test) = tf.keras.datasets.mnist.load_data()
images_train = np.reshape(images_train / 255.0, (len(images_train), len(images_train[0]) * len(images_train[0][0]))).T
images_test = np.reshape(images_test / 255.0, (len(images_test), len(images_test[0]) * len(images_test[0][0]))).T

def one_hot_encoding(data):
    encoded = np.zeros((len(data), max(data) - min(data) + 1))
    for idx in range(len(data)):
        encoded[idx][data[idx]] = 1
    return encoded.T

labels_train, labels_test = one_hot_encoding(labels_train), one_hot_encoding(labels_test)

class NeuralNetwork_2():
    def __init__(self, n0, n1, n2):
        self.weight1 = np.random.rand(n1, n0)
        self.weight2 = np.random.rand(n2, n1)
        self.bias1 = np.random.rand(n1)
        self.bias2 = np.random.rand(n2)
    def linear_hypo(self, w, x, b):
        return (np.matmul(w, x).T + b).T
    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))
    def relu(self, z):
        return np.maximum(z, 0)
    def softmax(self, z):
        return np.exp(z) / np.exp(z).sum(axis=0)
    def hypothesis(self, func, lh):
        if func == 'sigmoid':
            return self.sigmoid(lh)
        elif func == 'relu':
            return self.relu(lh)
        elif func == 'softmax':
            return self.softmax(lh)
        else:
            sys.exit('Error in hypothesis: There is no {} function'.format(func))
    def cost(self, y, hypo):
        # cross entropy
        sum = np.multiply(y, np.log(hypo)).sum(axis=0)
        return -np.average(sum)
    def train(self, features, labels, learning_rate, EPOCHS, batch_size):
        for iter in range(EPOCHS + 1):
            for batch in range(int(len(features[0]) / batch_size)):
                features_batch = features[:, batch * batch_size : (batch + 1) * batch_size]
                labels_batch = labels[:, batch * batch_size : (batch + 1) * batch_size]
                # layer1: relu, layer2: softmax, cost: cross-entropy
                z1 = self.linear_hypo(self.weight1, features_batch, self.bias1)
                layer1 = self.hypothesis('relu', z1)
                z2 = self.linear_hypo(self.weight2, layer1, self.bias2)
                layer2 = self.hypothesis('softmax', z2)
                # gradient
                dz2 = layer2 - labels_batch
                dw2 = np.matmul(dz2, layer1.T)
                db2 = dz2.sum(axis=1)

                dz1 = np.multiply(np.matmul(self.weight2.T, dz2), np.where(z1 > 0, 1, 0)) # relu gradient: if x > 0: 1 else: 0 => np.where(z1 > 0, 1, 0)
                dw1 = np.matmul(dz1, features_batch.T)
                db1 = dz1.sum(axis=1)

                self.weight2 = self.weight2 - dw2 * learning_rate
                self.weight1 = self.weight1 - dw1 * learning_rate
                self.bias2 = self.bias2 - db2 * learning_rate
                self.bias1 = self.bias1 - db1 * learning_rate

                error = self.cost(labels_batch, layer2)
            if iter % (EPOCHS / 10) == 0:
                print("iter: {:4} error: {:10.4f}".format(iter, error))
    def test_accuracy(self, features, labels):
        hypothesis = self.hypothesis('softmax', self.linear_hypo(self.weight2, self.hypothesis('relu', self.linear_hypo(self.weight1, features, self.bias1)), self.bias2))
        prob = np.average((np.argmax(hypothesis, axis=0) == np.argmax(labels, axis=0))) * 100
        print("Test Accuracy: {:10.4f}".format(prob))
model = NeuralNetwork_2(len(images_train), 5, 10)
model.train(images_train, labels_train, 0.0001, 100, 10)
model.test_accuracy(images_test, labels_test)

iter:    0 error:     8.2868
iter:   10 error:     0.7600
iter:   20 error:     0.6516
iter:   30 error:     0.5895
iter:   40 error:     0.5544
iter:   50 error:     0.5317
iter:   60 error:     0.5156
iter:   70 error:     0.5034
iter:   80 error:     0.4939
iter:   90 error:     0.4862
iter:  100 error:     0.4799
Test Accuracy:    87.7600
